In [15]:
import pandas as pd
data = pd.read_csv("menu.csv")
data.head()

,Name,Image,Ingredients,Price
0,Idly,./Imgs/pic2.jpeg,"Idli rice, Urad dal, Fenugreek seeds, Salt, Water",20
1,Dosa,./Imgs/pic2.jpeg,"Rice, Urad dal, Fenugreek seeds, Salt, Water, Oil",30


In [16]:
dishes = data.iloc[:, 0].to_numpy()
prices = data.iloc[:, -1].to_numpy()
menu = ""
for index, row in data.iterrows():
    ingred = row['Ingredients'].replace(",", "-")
    menu += f"{row['Name']}-{row['Image']}-{ingred}-{row['Price']}_"
menu = menu[:-1]
print(menu)

Idly-./Imgs/pic2.jpeg-Idli rice- Urad dal- Fenugreek seeds- Salt- Water-20_Dosa-./Imgs/pic2.jpeg-Rice- Urad dal- Fenugreek seeds- Salt- Water- Oil-30


In [17]:
import google.generativeai as genai
from dotenv import load_dotenv
import os
load_dotenv()

genai.configure(api_key=os.getenv('API_KEY'))

In [18]:
generation_config = {"temperature": 0.9, "top_p": 1, "top_k": 1, "max_output_tokens": 2048}

safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"}
]

txtModel = genai.GenerativeModel(model_name="gemini-1.0-pro",
generation_config=generation_config,
safety_settings=safety_settings)

imgModel = genai.GenerativeModel("gemini-pro-vision")


In [19]:
convo = txtModel.start_chat(history=[])
convo.send_message(f"Identify any spelling mistakes in the menu: {data}\n to accept or reject the menu with a valid reason\nNote:ouput should be in the format - \"accepted/rejected,no spelling mistakes found/spelling mistake found in dish_name: \"mistake\" should be \"correct spelling\"\"")
spelling = convo.last.text
print(spelling)

accepted, no spelling mistakes found


In [20]:
convo = txtModel.start_chat(history=[])
convo.send_message(f"Identify any incorrect ingredient present in the dishes in the menu: {data}\n to accept or reject the menu with a valid reason\n scale your confidence about your decision between 1 to 10\nNote:ouput should be in the format - \"confidence points, accepted/rejected, no ingredient mistakes found/ingredient mistake found in dish_name: \"mistake\" will not be used in the dish_name\"")
ingredients = convo.last.text
print(ingredients)

9, accepted, no ingredient mistakes found


In [21]:
convo = txtModel.start_chat(history=[])
cusineLst = []
vegLst = []
for index, row in data.iterrows():
    convo.send_message(f"Identify the cusine of the dish: {row['Name']}\nNote:ouput should be in one format - \"cusine\"")
    cusine = convo.last.text
    cusineLst.append(cusine)
    print(cusine)
    convo.send_message(f"Identify the whether the dish: {row['Name']} is vegetarian or not with respect to the ingredients: {row['Ingredients']}\nNote:ouput should be in one format - \"veg/non-veg\"")
    veg = convo.last.text
    vegLst.append(veg)
    print(veg)

South Indian
veg
South Indian
veg


In [22]:
from collections import Counter
def cusineEvaluator():
    ans = ""
    countDct = Counter(cusineLst)
    print(countDct)
    for key, val in countDct.items():
        if val == 1:
            ind = cusineLst.index(key)
            ans = f"rejected, the Cusine of {dishes[ind]} does not match with rest of the dishes"
            return ans
        if val <= len(dishes)/3:
            ind = cusineLst.index(key)
            ans = f"manual, the Cusine of {dishes[ind]} does not match with rest of the dishes"
            return ans
    ans = f"accepted, the Cusine of all dishes are matching"
    return ans
print(cusineEvaluator())

"""
Vada,./Imgs/pic1.jpeg,"Idli rice, Urad dal, Fenugreek seeds, Salt, Water",20
Pongal,./Imgs/pic2.jpeg,"Rice, Urad dal, Fenugreek seeds, Salt, Water, Oil",30
Poori,./Imgs/pic1.jpeg,"Idli rice, Urad dal, Fenugreek seeds, Salt, Water",20
"""

Counter({'South Indian': 2})
accepted, the Cusine of all dishes are matching


'\nVada,./Imgs/pic1.jpeg,"Idli rice, Urad dal, Fenugreek seeds, Salt, Water",20\nPongal,./Imgs/pic2.jpeg,"Rice, Urad dal, Fenugreek seeds, Salt, Water, Oil",30\nPoori,./Imgs/pic1.jpeg,"Idli rice, Urad dal, Fenugreek seeds, Salt, Water",20\n'

In [23]:
def vegEvaluator():
    ans = ""
    countDct = Counter(vegLst)
    print(countDct)
    for key, val in countDct.items():
        if val == 1:
            ind = vegLst.index(key)
            ans = f"rejected, this {dishes[ind]} is {key} and not matching with rest of the dishes"
            return ans
        if val <= len(dishes)/3:
            ind = vegLst.index(key)
            ans = f"manual, this {dishes[ind]} is {key} and not matching with rest of the dishes"
            return ans
    ans = f"accepted, the veg of all dishes are matching"
    return ans
print(vegEvaluator())

Counter({'veg': 2})
accepted, the veg of all dishes are matching


In [24]:
for i in range(len(dishes)):
    convo.send_message(f"Is {prices[i]} rupees for the dish: {dishes[i]} is (cheap, normal, overpriced)\n to accept or reject the dish with a valid reason\n scale your confidence about your decision between 1 to 10\nNote:ouput should be in the format - \"confidence points, accepted/rejected, reason\"")
    pricing = convo.last.text
    print(pricing)

8, cheap, Idly is a staple breakfast food in South India and is typically very affordable. 20 rupees is a reasonable price for a plate of idlis.
7, normal, Dosa is a popular South Indian dish that is typically served with chutney and sambar. 30 rupees is a fair price for a dosa, especially if it is made with good quality ingredients. However, the price may be considered slightly overpriced in some areas.


In [38]:
import PIL.Image

for index, row in data.iterrows():
    img = PIL.Image.open(row["Image"])
    imgMistakeResponse = imgModel.generate_content([f"what is the dish in the image\n scale your confidence about your dish identification between 1 to 100\nNote:ouput should be in the format - \"<confidence points>,<dish_name>", img])
    imgDish = imgMistakeResponse.text
    imgDish = imgDish.split(",")
    print(imgDish)
    convo.send_message(f"Is {imgDish[1]} and {dishes[index]} same?\n to accept or reject the dish with a valid reason if they are same\n scale your confidence about your decision between 1 to 10\nNote:ouput should be in the format - \"<confidence points>, accepted/rejected, <dish and the image are same>/mistake reason\"")
    imgDishMistake = convo.last.text
    print(imgDishMistake)
    qualityResponse = imgModel.generate_content([f"Based on the visual cues analyze the given image for hygiene, color, texture and consistency.\n to accept or reject the dish with a valid reason\n scale your confidence about your dish identification between 1 to 100\nNote:ouput should be in the format- \"<confidence points>, accepted/rejected, <no image mistakes found>/mistake reason", img])
    quality = qualityResponse.text
    print(quality)


[' 90', 'Masala Dosa']
90, rejected, Masala Dosa and Idly are not the same dish. Masala dosa is a type of dosa that is filled with a potato and onion mixture, while idli is a steamed cake made from a batter of fermented rice and lentils. While both dishes are popular in South India, they have different textures, flavors, and shapes.
 80, accepted, no image mistakes found
[' 90', ' Masala Dosa']
90, accepted, Masala Dosa and Dosa are the same dish. Masala dosa is a type of dosa that is filled with a potato and onion mixture. It is one of the most popular dishes in South India.
 80, accepted, no image mistakes found
